In [2]:
#! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain youtube-transcript-api pytube

In [7]:
import os
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_ENDPOINT'] = 'https://api.smith.langchain.com'
os.environ['LANGCHAIN_API_KEY'] = "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"

In [8]:
import os

#Setting Environment variable
os.environ["AZURE_OPENAI_API_VERSION"] = "2023-07-01-preview"
os.environ["AZURE_OPENAI_ENDPOINT"] = 'https://dskumar.openai.azure.com/'
os.environ["AZURE_OPENAI_API_KEY"] ="62855d6dd08945819bf83aee0c104127"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] ="DskumarDeployment"
os.environ['OPENAI_TYPE']="Azure"
os.environ["LLM_MODEL"] = "gpt-35-turbo-16k"
os.environ["LLM_EMBEDDING_MODEL"] = "dskumar-text-embedding-ada-002"

from langchain_core.messages import AIMessage,SystemMessage,HumanMessage
from langchain_openai import AzureChatOpenAI



In [9]:
from langchain_openai import AzureOpenAIEmbeddings ##embedding type
embeddings = AzureOpenAIEmbeddings(model = os.environ["LLM_EMBEDDING_MODEL"],)

In [12]:
from langchain_community.document_loaders.pdf import PyPDFLoader
loader = PyPDFLoader(r".\citi-2022-annual-report.pdf")
output=loader.load()
print(output[0].page_content)

2022 ANNUAL REPORT
Citi is proud of its 
colleagues in Ukraine, 
who are supporting  
the country through  
a devastating war.


In [54]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

#loader = WebBaseLoader("https://docs.smith.langchain.com/overview")
docs = loader.load()
documents = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200
).split_documents(docs)
vector = FAISS.from_documents(documents, embeddings,
)
tenk_retriever = vector.as_retriever()

# Logical Routing

In [13]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field

# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    datasource: Literal["tenk_docs", "Process_docs"] = Field(
        ...,
        description="Given a user question choose which datasource would be most relevant for answering their question",
    )

# LLM with function call 
azurechatmodel = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
)
structured_llm = azurechatmodel.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the department the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

C:\Users\JANU\AppData\Roaming\Python\Python311\site-packages\langchain_core\_api\beta_decorator.py:87: LangChainBetaWarning: The function `with_structured_output` is in beta. It is actively being worked on, so the API may change.
  warn_beta(


In [59]:
router

ChatPromptTemplate(input_variables=['question'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are an expert at routing a user question to the appropriate data source.\n\nBased on the department the question is referring to, route it to the relevant data source.')), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], template='{question}'))])
| RunnableBinding(bound=AzureChatOpenAI(client=<openai.resources.chat.completions.Completions object at 0x0000020F97161C10>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x0000020F955E13D0>, openai_api_key=SecretStr('**********'), openai_proxy='', azure_endpoint='https://dskumar.openai.azure.com/', deployment_name='DskumarDeployment', openai_api_version='2023-07-01-preview', openai_api_type='azure'), kwargs={'tools': [{'type': 'function', 'function': {'name': 'RouteQuery', 'description': 'Route a user query to the most relevant datasource.', 'p

In [60]:
question = """what is the finacial statement of the company?"""

result = router.invoke({"question": question})

In [61]:
result

RouteQuery(datasource='tenk_docs')

In [62]:
question = """what is the finacial statement of the company?

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("english")
"""

result = router.invoke({"question": question})

In [63]:
result

RouteQuery(datasource='tenk_docs')

In [64]:
# Define your retrievers
retrievers = {
    "tenk_docs": "tenk_retriever",
    "Process_docs": "Process_docs_retriever"
}

In [36]:
# Updated choose_route function
def choose_route(result):
    if "10K_docs" in result.datasource:
        return retrievers["10K_docs"]
    elif "process_docs" in result.datasource:
            
    else:
        return "No logical docs"

from langchain_core.runnables import RunnableLambda

full_chain = router | RunnableLambda(choose_route)

In [38]:
retriver = full_chain.invoke({"question": question})

In [39]:
retriver

'tenk_retriever'

In [41]:
from langchain.tools.retriever import create_retriever_tool

In [46]:
retriever_tool = create_retriever_tool(name='retriever',description="retriever", retriever=tenk_retriever)
tools = [retriever_tool]

In [53]:
from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(azurechatmodel, tools, prompt=prompt)


ValueError: Prompt missing required variables: {'agent_scratchpad'}

## Sematic routing

In [1]:
from langchain.utils.math import cosine_similarity
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
#from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [5]:
# Two prompts
template_10K = """You are a very smart at reading finacial statement of a company. \
You are great at answering questions about 10K document in a concise and easy to understand manner. \
When you don't know the answer to a question you admit that you don't know.

Here is a question:
{query}"""

process_template = """You are a very good at process management. You are great at answering process management questions. \
You are so good because you are able to break down hard problems into their component parts, \
answer the component parts, and then put them together to answer the broader question.

Here is a question:
{query}"""

In [10]:
# Embed prompts
from langchain_openai import AzureOpenAIEmbeddings ##embedding type
embeddings = AzureOpenAIEmbeddings(model = os.environ["LLM_EMBEDDING_MODEL"],)
prompt_templates = [template_10K, process_template]
prompt_embeddings = embeddings.embed_documents(prompt_templates)

In [11]:
# Route question to prompt 
def prompt_router(input):
    # Embed question
    query_embedding = embeddings.embed_query(input["query"])
    # Compute similarity
    similarity = cosine_similarity([query_embedding], prompt_embeddings)[0]
    most_similar = prompt_templates[similarity.argmax()]
    # Chosen prompt 
    print("Using template_10K" if most_similar == template_10K else "Using process")
    return PromptTemplate.from_template(most_similar)

In [16]:
chain = (
    {"query": RunnablePassthrough()}
    | RunnableLambda(prompt_router)
    | azurechatmodel
    | StrOutputParser()
)

print(chain.invoke("summaries the annual report?"))

Using template_10K
Summarizing the annual report involves condensing the key information and highlights of a company's financial performance and operations throughout the year. It typically includes a summary of the company's financial statements, such as the balance sheet, income statement, and cash flow statement, along with any significant events or developments that may have influenced the company's performance. The annual report summary aims to provide a concise overview of the company's financial health, strategic goals, major achievements, and potential risks or challenges. It is often presented in a narrative format, highlighting the most important aspects for investors and stakeholders to understand the company's overall performance and prospects.
